In [1]:
import util
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# load data
PERCENT_OF_DATA = 0.0125
train_loader = util.load_data.load_dataset('Train', batch_size=32, data_percentage=PERCENT_OF_DATA)
test_loader = util.load_data.load_dataset('Test', batch_size=32, data_percentage=PERCENT_OF_DATA)
validation_loader = util.load_data.load_dataset('Validation', batch_size=32, data_percentage=PERCENT_OF_DATA)

In [4]:
num_classes = 2  # Binary classification: real or fake
model = util.dfnn_model.DeepfakeDetectorCNN(num_classes=num_classes)

# Example training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
util.train_model.train_with_validation(model, train_loader, validation_loader, criterion, optimizer, device)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10] - Training:


100%|████████████████████████████████████████████████████████| 55/55 [02:12<00:00,  2.42s/it]


Training Loss: 0.3372
Validation Loss: 0.5854, Validation Accuracy: 81.30%

Epoch [2/10] - Training:


100%|████████████████████████████████████████████████████████| 55/55 [02:04<00:00,  2.26s/it]


...

Training Loss: 0.0554
Validation Loss: 0.2085, Validation Accuracy: 93.50%

Epoch [9/10] - Training:


100%|████████████████████████████████████████████████████████| 55/55 [02:06<00:00,  2.30s/it]


Training Loss: 0.0658
Validation Loss: 0.3108, Validation Accuracy: 90.45%

Epoch [10/10] - Training:


100%|████████████████████████████████████████████████████████| 55/55 [02:06<00:00,  2.30s/it]


Training Loss: 0.0626
Validation Loss: 0.4492, Validation Accuracy: 87.20%



In [6]:
test_loss, test_accuracy = util.train_model.test(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%\n")

Test Loss: 0.5993, Test Accuracy: 82.35%

